In [1]:
import os
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
from tools.benchmarks.helpers import *
import pickle
import datetime
import matplotlib.ticker as ticker
import math
from scipy.stats import linregress
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot
from scipy.optimize import curve_fit

In [ ]:
home = os.path.expanduser("~") 
os.chdir(str(project_dir))

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/operational-benchmarking/op-bench-runtime"
def get_write_time (algorithm, Ntx, Nty, target, nt, nr, batching):
    command = f"{base_cmd} {algorithm} {Ntx} {Nty} {target} {nt} {nr} {batching}"
    return os.popen(command).read()

In [ ]:
trials = 1000
reps = 1
atoms_dim = [(2,2), (4,4), (8,8), (12,12), (16,16), (24,24), (32,32)]
traps_dims = [(2,4), (4,8), (8,14), (12, 20), (16, 26), (24, 40), (32, 54)]
algorithms = ["REDREC-CPU-V3-2D"]

for algorithm in algorithms:
    time_data  = []
    for i in range(len(atoms_dim)):
        print(f"{i}/{len(atoms_dim)}")
        write_time_list = []
        write_time_str = get_write_time(algorithm, traps_dims[i][0], traps_dims[i][1], atoms_dim[i][0] * atoms_dim[i][1], trials, reps, 0)
        time_data += [(float(write_time_str.split(',')[0]), float(write_time_str.split(',')[1]))]
    # Saving the results
    file_name = "timing-batching-" + algorithm + "-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
    with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name), "wb") as file:
        pickle.dump(
            {
                "xvals": [x * y for (x, y) in atoms_dim],
                "algorithm": algorithm,
                "experiment": file_name,
                "data": time_data
            }, file
        )


In [ ]:
trials = 1000
reps = 1
traps_dims = [8,16,32,64,128,256,512,768,1024] 
atoms_dims = [int(x*0.6) for x in traps_dims]

algorithms = ["LINEAR-EXACT-GPU-V2-1D","LINEAR-EXACT-V2-1D"]

for algorithm in algorithms:
    time_data  = []
    for i in range(len(atoms_dims)):
        print(f"{i}/{len(atoms_dims)}")
        write_time_list = []
        write_time_str = get_write_time(algorithm, 1, traps_dims[i], atoms_dims[i], trials, reps)
        time_data += [(float(write_time_str.split(',')[0]), float(write_time_str.split(',')[1]))]
    # Saving the results
    file_name = "timing-" + algorithm + "-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
    with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name), "wb") as file:
        pickle.dump(
            {
                "xvals": atoms_dim,
                "algorithm": algorithm,
                "experiment": file_name,
                "data": time_data
            }, file
        )

# Load results

In [ ]:
file_name_cpu = "timingREDREC-CPU-V3-2D-varying-traps-2024-07-31, 19-04-58.pickle"
file_name_gpu = "timingREDREC-GPU-V3-2D-varying-traps-2024-07-31, 19-04-02.pickle"
file_name_bird = "timingBIRD-CPU-2D-varying-traps-2024-07-31, 19-05-56.pickle"
file_name_aro = "timingARO-2D-varying-traps-2024-08-01, 04-31-20.pickle"
file_name_cpu_batching = "timing-batching-REDREC-CPU-V3-2D-varying-traps-2024-08-01, 09-09-05.pickle"

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_cpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_cpu = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_gpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_gpu = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_bird), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_bird = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_aro), "rb") as file:
    data = pickle.load(file)
    atoms_aro = data["xvals"]
    time_data_aro = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_cpu_batching), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_cpu_batching = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_gpu_batching), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_gpu_batching = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_bird_batching), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_bird_batching = data["data"]

In [ ]:
def func(x, a, b, c):
  return  c * x ** 2 

def func_cubic(x, a, b, c, d):
  return d * x ** 3  + a

def func_quartic(x, a, b, c, d, e):
  return e * x ** 4 + a

def func_eight(x, e, a):
  return e * x ** 8 + a

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]

xvals_batching = [2,4,8,12,16,24,32] 
yvals_cpu_batching = [x[0]/1e3 for x in time_data_cpu_batching]
yerr_cpu_batching = [x[1]/1e3 for x in time_data_cpu_batching]


fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (us)")
model_cpu, _ = curve_fit(func_cubic, xvals, yvals_cpu)
model_cpu_batching, _ = curve_fit(func_cubic, xvals_batching, yvals_cpu_batching)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**3 * model_cpu[3] + model_cpu[0] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**3 * model_cpu_batching[3] + model_cpu_batching[0]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E06900", "lw": 1}
).draw()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals_batching], [yvals_cpu, yvals_cpu_batching], yerr_list=[yerr_cpu, yerr_cpu_batching], markers=mark(["rrccpu", "rrcbatch"]), palette=pal(["rrccpu", "rrcbatch"]),  save_path=str(project_dir) + "/cpu-v-gpu-runtime-2d-batching.pdf")
ax.legend(["Without batching", "With batching"],fontsize=4)
yticks(ax,[0,300,600,900,1200])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("cpu-v-gpu-runtime-2d-batching.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]
yvals_gpu = [x[0]/1e3 for x in time_data_gpu]
yerr_gpu = [x[1]/1e3 for x in time_data_gpu]
yvals_bird = [x[0]/1e3 for x in time_data_bird]
yerr_bird = [x[1]/1e3 for x in time_data_bird]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (us)")
model_gpu = np.poly1d(np.polyfit(xvals, yvals_gpu, 1))
model_bird, _ = curve_fit(func_cubic, xvals, yvals_bird)
model_cpu, _ = curve_fit(func_cubic, xvals, yvals_cpu)
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals], [yvals_cpu, yvals_gpu, yvals_bird], yerr_list= [yerr_cpu, yerr_gpu, yerr_bird],markers=mark(["rrccpu", "rrcgpu", "bird"]), palette=pal(["rrccpu", "rrcgpu","bird"]), save_path=str(project_dir) + "/bird-rrccpu-rrcgpu-runtime-2d.pdf")
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**3 * model_cpu[3] + model_cpu[0] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_gpu.coefficients[0] + model_gpu.coefficients[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**3 * model_bird[3]  + model_bird[0] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#648EF7", "lw": 1}
).draw()


ax.legend(["Red-rec CPU", "Red-rec GPU", "Bird CPU"],fontsize=4)
yticks(ax, [0,60, 120,180, 240])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("bird-rrccpu-rrcgpu-runtime-2d.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]
yvals_gpu = [x[0]/1e3 for x in time_data_gpu]
yerr_gpu = [x[1]/1e3 for x in time_data_gpu]


fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (us)")
model_gpu = np.poly1d(np.polyfit(xvals, yvals_gpu, 1))
model_cpu, _ = curve_fit(func_cubic, xvals, yvals_cpu)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**3 * model_cpu[3] + model_cpu[0]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_gpu.coefficients[0] + model_gpu.coefficients[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()

prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yvals_cpu, yvals_gpu], yerr_list=[yerr_cpu, yerr_gpu],  markers=mark(["rrccpu", "rrcgpu"]), palette=pal(["rrccpu", "rrcgpu"]),save_path=str(project_dir) + "/cpu-v-gpu-runtime-2d.pdf")

ax.legend(["Red-rec CPU", "Red-rec GPU"],fontsize=4)
yticks(ax, [0,60, 120,180, 240])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("cpu-v-gpu-runtime-2d.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = [math.log(x[0]/1e9, 10) for x in time_data_cpu]
yvals_bird = [math.log(x[0]/1e9, 10) for x in time_data_bird]
yvals_aro = [math.log(x[0]/1e9, 10) for x in time_data_aro]
yvals_gpu = [math.log(x[0]/1e9, 10) for x in time_data_gpu]



fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (s)")

model_cpu, _ = curve_fit(func_cubic, xvals, [x[0]/1e3 for x in time_data_cpu])
model_bird, _ = curve_fit(func_cubic, xvals, [x[0]/1e3 for x in time_data_bird])
model_gpu = np.poly1d(np.polyfit(xvals, yvals_gpu, 1))
model_aro, _ = curve_fit(func_eight, xvals, [x[0]/1e3 for x in time_data_aro])
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [math.log((x**3 * model_cpu[3] + model_cpu[0] )/1e6, 10) for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_gpu.coefficients[0] + model_gpu.coefficients[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#57B52C", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [math.log((x**3 * model_bird[3] + model_bird[0]) /1e6, 10) for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#648EF7", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [math.log((x**8 * model_aro[0] + model_aro[1]) /1e6, 10) for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()

prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals, xvals], [yvals_cpu, yvals_gpu, yvals_bird, yvals_aro],  markers=mark(["rrccpu", "rrcgpu", "bird", "aro"]), palette=pal(["rrccpu", "rrcgpu", "bird", "aro"]),save_path=str(project_dir) + "/all-runtime-2d.pdf")



ax.legend(["Red-rec CPU","Red-rec GPU", "Bird", "ARO"],fontsize=4)
yticks(ax, [-6, -4,-2,0,2])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))

plt.savefig("all-runtime-2d.pdf", format='pdf')
plt.show()

In [ ]:
xvals = [2,4,8,12,16,24,32] 
yvals_cpu = []
yerr_cpu = []
yvals_gpu = []
yerr_gpu = []
yvals_bird = []
yerr_bird = []
yvals_aro = [x[0]/1e9 for x in time_data_aro]
yerr_aro = [x[1]/1e9 for x in time_data_aro]

xvals = [2,4,8,12,16,24,32] 
yvals_cpu = []
yerr_cpu = []
yvals_gpu = []
yerr_gpu = []
yvals_bird = []
yerr_bird = []
yvals_aro = [x[0]/1e9 for x in time_data_aro]
yerr_aro = [x[1]/1e9 for x in time_data_aro]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Time (s)")

model_aro, _ = curve_fit(func_eight, xvals, yvals_aro)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**8 * model_aro[0] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#C46BC7", "lw": 1}
).draw()

prod_plot.production_quality_scatter(fig, ax, [xvals], [yvals_aro], yerr_list=[yerr_aro], markers=mark(["aro"]), palette=pal(["aro"]), save_path=str(project_dir) + "/aro.pdf")

ax.legend(["ARO"],fontsize=4)
ax.set_xlim(-1, 33)
ax.set_ylim(-2, 22)
yticks(ax, [0,8,16,24, 32])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("aro.pdf", format='pdf')
plt.show()
print (model_aro)

In [ ]:
file_name_cpu = "timing-LINEAR-EXACT-V2-1D-varying-traps-2024-08-01, 09-23-17.pickle"
file_name_gpu = "timing-LINEAR-EXACT-GPU-V2-1D-varying-traps-2024-08-01, 09-22-51.pickle"

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_cpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_cpu = data["data"]

with open(os.path.join(home, "LLRS", "resources", "solver-runtime", file_name_gpu), "rb") as file:
    data = pickle.load(file)
    atoms = data["xvals"]
    time_data_gpu = data["data"]


In [ ]:
xvals = [8,16,32,64,128,256,512,768,1024] 
yvals_cpu = [x[0]/1e3 for x in time_data_cpu]
yvals_gpu = [x[0]/1e3 for x in time_data_gpu]
yerr_cpu = [x[1]/1e3 for x in time_data_cpu]
yerr_gpu = [x[1]/1e3 for x in time_data_gpu]
fig, ax = plt.subplots() 
ax.set_xlabel("Number of traps")
ax.set_ylabel("Time (us)")
def func(x, a, b):
  return a * x**2 +b
result_cpu,_ = curve_fit(func, xvals, yvals_cpu)
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yvals_cpu, yvals_gpu], yerr_list=[yerr_cpu, yerr_gpu], markers=mark(["lincpu", "lingpu"]),palette=pal(["lincpu", "lingpu"]),  save_path=str(project_dir) + "/cpu-v-gpu-runtime-1d.pdf")

PlotPlotter(
    fig,
    ax,
    range(1025),
    [result_cpu[0]* x**2 + result_cpu[1] for x in range(1025)],
    style = {"linestyle" : "-", "color": "#A9A9A9", "lw": 1}
).draw()



ax.legend(["Linear exact CPU", "Linear exact GPU"],fontsize=4)
yticks(ax, [0, 8,16,24,32])
xticks(ax, [0, 256, 512, 768, 1024])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("cpu-v-gpu-runtime-1d.pdf", format='pdf')
plt.show()